In [1]:
pip install pytaxize

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Chargement des librairies

from pytaxize import Ids
import warnings
import sys
from pytaxize.itis import hierarchy_full
from pytaxize import scicomm
import pandas as pd
# Ca ne marche pas
# from pytaxize.ncbi import hierarchy
# from pytaxize import Classification

In [3]:
def hierarchy(ids):
    """
    Get a full taxonomic hierarchy from NCBI

    :param ids: one or more NCBI taxonomy ids

    :note: Remember to set your Entrez API key as `ENTREZ_KEY`

    :return: dict, named with ids given to `ids`, 
        where each value in the dict is a list of taxa, each
        a dict with the fields ``ScientificName``, ``Rank``, and ``TaxId``

    Usage::

        from pytaxize import ncbi
        ncbi.hierarchy(ids=9606)
        ncbi.hierarchy(ids=[9606,55062,4231])
    """
    toget = ["ScientificName", "Rank", "TaxId"]
    key = os.environ.get("ENTREZ_KEY")
    if key is None:
        raise Exception("ENTREZ_KEY is not defined")
    if not isinstance(ids, list):
        ids = [ids]
    idz = ",".join([str(x) for x in ids])
    args = {"db": "taxonomy", "ID": idz, "api_key": key}
    res = _entrez("efetch", args)
    taxa = res.xpath("//TaxaSet/Taxon")
    out = []
    for x in range(len(taxa)):
        nodes = taxa[x].xpath(".//LineageEx/Taxon")
        tmp = [
            dict(zip(toget, [node.xpath(w)[0].text for w in toget])) for node in nodes
        ]
        tmp.append(dict(zip(toget, [taxa[x].xpath(w)[0].text for w in toget])))
        out.append(tmp)
    return dict(zip(ids, out))

class Classification(object):
    """
    Classification: Retrieve taxonomic hierarchy for taxonomic IDs

    Usage::

        from pytaxize import Classification
        
        # ITIS
        ## one id
        x = Classification(99208)
        x
        x.ids
        res = x.itis()
        res[99208]
        
        ## many ids - with one invalid id
        x = Classification([99208, 129313, 9999999])
        x
        x.ids
        res = x.itis()
        res[99208]
        res[129313]
        res[9999999]

        # NCBI
        x = Classification(9606)
        x
        x.ids
        x.ncbi()
    """

    def __init__(self, ids):
        if isinstance(ids, int):
            ids = [ids]
        self.ids = ids

    def __repr__(self):
        x = """<%s>\n""" % type(self).__name__
        y = """  ids: %s""" % ",".join([str(w) for w in self.ids[:10]])
        return x + y

    def itis(self):
        out = []
        for i in range(len(self.ids)):
            id = self.ids[i]
            res = hierarchy_full(id)
            if res[0] is None:
                warnings.warn("No results for taxon '" + str(id) + "'")
                res = {}
            out.append(res)
        out = dict(zip(self.ids, out))
        return out

    def ncbi(self):
        res = hierarchy(self.ids)
        # out = []
        # for i in range(len(self.ids)):
        #     id = self.ids[i]
            # if res[0] is None:
            #     warnings.warn("No results for taxon '" + str(id) + "'")
            #     res = {}
            # out.append(res)
        # out = dict(zip(self.ids, out))
        return res

In [100]:
# Peut être utile pour obtenir les autres noms d'une espèce
x = Ids('Geomyza')
print("itis : ", x.itis())
print("ids : ", x.ids)
scicomm.sci2comm(x)

itis :  None
ids :  {'Geomyza': [{'id': '146647', 'name': 'Geomyza unipunctum', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=146647'}, {'id': '146886', 'name': 'Geomyza griseola', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=146886'}]}


{'Geomyza': []}

In [103]:
# Enregistrement du nom de l'espèce
nom = Ids('Geomyza unipunctum')
nom.itis()
print("ids :", nom.ids)
# Extraction de l'id de l'espèce
for item in nom.ids[nom.name[0]]:
    if item['name'] == nom.name[0]:
        id = item['id']
        break
id = int(id)
print("id :", id)
# Obtenir la classification de l'espèce
code = hierarchy_full(id, as_dataframe=True)

ids : {'Geomyza unipunctum': [{'id': '146647', 'name': 'Geomyza unipunctum', 'rank': 'species', 'uri': 'https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=146647'}]}
id : 146647


In [104]:
code

,author,parentName,parentTsn,rankName,taxonName,tsn
0,"Zetterstedt, 1847",,,Species,Geomyza unipunctum,146647


In [83]:
hierarchy_full(tsn = 139923, as_dataframe=True)

,author,parentName,parentTsn,rankName,taxonName,tsn
0,"Wulp, 1867",,,Species,Syrphus signatus,139923


In [7]:
# Obtention de l'ordre, de la famille et du genre
hierarchie = lignee[id]
for item in hierarchie:
    if item['rankName'] == 'Order':
        order_taxon_name = item['taxonName']
    if item['rankName'] == 'Family':
        family_taxon_name = item['taxonName']
    if item['rankName'] == 'Genus':
        genus_taxon_name = item['taxonName']

print("Ordre :", order_taxon_name)
print("Famille :", family_taxon_name)
print("Genre :", genus_taxon_name)

Ordre : Hymenoptera
Famille : Apidae
Genre : Apis


# Pipeline complète

## **Apis mellifera**

In [8]:
# Enregistrement du nom de l'espèce
nom = Ids('Apis mellifera')
nom.itis()
# Extraction de l'id de l'espèce
for item in nom.ids[nom.name[0]]:
    if item['name'] == nom.name[0]:
        id = item['id']
        break
id = int(id)
# Obtenir la classification de l'espèce
code = Classification(id)
lignee = code.itis()
# Obtention de l'ordre, de la famille, du genre et de l'espèce
for item in lignee[id]:
    if item['rankName'] == 'Order':
        order_taxon_name = item['taxonName']
    if item['rankName'] == 'Family':
        family_taxon_name = item['taxonName']
    if item['rankName'] == 'Genus':
        genus_taxon_name = item['taxonName']
    if item['rankName'] == 'Species':
        species_taxon_name = item['taxonName']
# Création d'un tableau avec les informations
df = pd.DataFrame({'Ordre': [order_taxon_name], 
                   'Famille': [family_taxon_name], 
                   'Genre': [genus_taxon_name], 
                   'Espèce': [species_taxon_name]})
df

,Ordre,Famille,Genre,Espèce
0,Hymenoptera,Apidae,Apis,Apis mellifera


In [9]:
# Enregistrement du nom de l'espèce
nom = Ids('Apis mellifera')
nom.itis()
# Extraction de l'id de l'espèce
for item in nom.ids[nom.name[0]]:
    if item['name'] == nom.name[0]:
        id = item['id']
        break
id = int(id)
# Obtenir la classification de l'espèce
code = Classification(id)
lignee = code.itis()
# Obtention de l'ordre, de la famille, du genre et de l'espèce
for item in lignee[id]:
    if item['rankName'] == 'Order':
        order_taxon_name = item['taxonName']
    if item['rankName'] == 'Family':
        family_taxon_name = item['taxonName']
    if item['rankName'] == 'Genus':
        genus_taxon_name = item['taxonName']
    if item['rankName'] == 'Species':
        species_taxon_name = item['taxonName']
# Création d'un tableau avec les informations
df1 = pd.DataFrame({'Ordre': [order_taxon_name], 
                   'Famille': [family_taxon_name], 
                   'Genre': [genus_taxon_name], 
                   'Espèce': [species_taxon_name]})
df1

,Ordre,Famille,Genre,Espèce
0,Hymenoptera,Apidae,Apis,Apis mellifera


## **Bombus**

In [10]:
# Enregistrement du nom de l'espèce
nom = Ids('Bombus')
nom.itis()
# Extraction de l'id de l'espèce
for item in nom.ids[nom.name[0]]:
    if item['name'] == nom.name[0]:
        id = item['id']
        break
id = int(id)
# Obtenir la classification de l'espèce
code = Classification(id)
lignee = code.itis()
# Obtention de l'ordre, de la famille, du genre et de l'espèce
for item in lignee[id]:
    if item['rankName'] == 'Order':
        order_taxon_name = item['taxonName']
    elif item['rankName'] == 'Family':
        family_taxon_name = item['taxonName']
    elif item['rankName'] == 'Genus':
        genus_taxon_name = item['taxonName']
    elif item['rankName'] == 'Species':
        species_taxon_name = item['taxonName']
# Création d'un tableau avec les informations
new_data = {'Ordre': order_taxon_name, 
            'Famille': family_taxon_name, 
            'Genre': genus_taxon_name, 
            'Espèce': species_taxon_name}
df = df.append(new_data, ignore_index=True)
df

C:\Users\corne\AppData\Local\Temp\ipykernel_18468\709395751.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data, ignore_index=True)


,Ordre,Famille,Genre,Espèce
0,Hymenoptera,Apidae,Apis,Apis mellifera
1,Hymenoptera,Apidae,Bombus,Bombus zonatus


## **Tettigonioidea** (écrit Tettigonoidea dans le tableau)

In [11]:
# Enregistrement du nom de l'espèce
nom = Ids('Tettigonioidea')
nom.itis()
# Extraction de l'id de l'espèce
for item in nom.ids[nom.name[0]]:
    if item['name'] == nom.name[0]:
        id = item['id']
        break
id = int(id)
# Obtenir la classification de l'espèce
code = Classification(id)
lignee = code.itis()
print(f"La classification de {nom.name[0]} est :")
lignee[id]

La classification de Tettigonioidea est :


[{'author': None,
  'parentName': '',
  'parentTsn': '',
  'rankName': 'Kingdom',
  'taxonName': 'Animalia',
  'tsn': '202423'},
 {'author': None,
  'parentName': 'Animalia',
  'parentTsn': '202423',
  'rankName': 'Subkingdom',
  'taxonName': 'Bilateria',
  'tsn': '914154'},
 {'author': None,
  'parentName': 'Bilateria',
  'parentTsn': '914154',
  'rankName': 'Infrakingdom',
  'taxonName': 'Protostomia',
  'tsn': '914155'},
 {'author': None,
  'parentName': 'Protostomia',
  'parentTsn': '914155',
  'rankName': 'Superphylum',
  'taxonName': 'Ecdysozoa',
  'tsn': '914158'},
 {'author': None,
  'parentName': 'Ecdysozoa',
  'parentTsn': '914158',
  'rankName': 'Phylum',
  'taxonName': 'Arthropoda',
  'tsn': '82696'},
 {'author': None,
  'parentName': 'Arthropoda',
  'parentTsn': '82696',
  'rankName': 'Subphylum',
  'taxonName': 'Hexapoda',
  'tsn': '563886'},
 {'author': None,
  'parentName': 'Hexapoda',
  'parentTsn': '563886',
  'rankName': 'Class',
  'taxonName': 'Insecta',
  'tsn': '9

In [12]:
# Obtention de l'ordre, de la famille, du genre et de l'espèce
for item in lignee[id]:
    if item['rankName'] == 'Order':
        order_taxon_name = item['taxonName']
    elif item['rankName'] == 'Family':
        family_taxon_name = item['taxonName']
    elif item['rankName'] == 'Genus':
        genus_taxon_name = item['taxonName']
    elif item['rankName'] == 'Species':
        species_taxon_name = item['taxonName']
# Création d'un tableau avec les informations
new_data = {'Ordre': order_taxon_name, 
            'Famille': family_taxon_name, 
            'Genre': genus_taxon_name, 
            'Espèce': species_taxon_name}
df = df.append(new_data, ignore_index=True)
df

C:\Users\corne\AppData\Local\Temp\ipykernel_18468\3446007282.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data, ignore_index=True)


,Ordre,Famille,Genre,Espèce
0,Hymenoptera,Apidae,Apis,Apis mellifera
1,Hymenoptera,Apidae,Bombus,Bombus zonatus
2,Orthoptera,Tettigoniidae,Bombus,Bombus zonatus


Deux problèmes :
- les fautes d'orthographes sont rédhibitoires
> Chercher dans la littérature s'il y a des moyens de chercher les noms similaires
- les catégories à extraires sont difficiles selon ce qui est disponible ;
    - si on a le nom complet il n'y a pas de problème : voir Apis mellifera
    - si on a le genre : les fonctions vont quand même chercher à nous donner le nom d'une espèce (donne la valeur +1)
    > Je dois essayer de corriger ça  
    > C'est corrigé !
    - si on a le nom d'une superfamille, la classification ne se poursut que jusqu'à +1 (Famille) donc on obtient dans le tableau le genre et l'espèce précédente (ancienne valeur enregistrée)
    > Créer une fonction pour donner les nouvelles valeurs  
    > Ca marche !

In [13]:
tableau = pd.DataFrame(columns=['Ordre', 'Super famille', 'Famille', 'Sous famille', 'Genre', 'Espèce'])

def get_classification(name):
    """
    """
    # Enregistrement du nom de l'espèce
    nom = Ids(name)
    nom.itis()
    # Extraction de l'id de l'espèce
    for item in nom.ids[nom.name[0]]:
        if item['name'] == nom.name[0]:
            id = int(item['id'])
            break
    # Obtenir la classification de l'espèce
    code = Classification(id)
    lignee = code.itis()
    # Obtention de l'ordre, de la famille, du genre et de l'espèce
    order_name, super_family_name, family_name, sub_family_name, genus_name, species_name = '', '', '', '', '', ''
    for item in lignee[id]:
        if item['parentName'] != nom.name[0]:
            if item['rankName'] == 'Order':
                order_name = item['taxonName']
            elif item['rankName'] == 'Superfamily':
                super_family_name = item['taxonName']
            elif item['rankName'] == 'Family':
                family_name = item['taxonName']
            elif item['rankName'] == 'Subfamily':
                sub_family_name = item['taxonName']
            elif item['rankName'] == 'Genus':
                genus_name = item['taxonName']
            elif item['rankName'] == 'Species':
                species_name = item['taxonName']
        else:
            break
    # Création d'un tableau avec les informations
    new_data = {'Ordre': order_name, 
                'Super famille': super_family_name,
                'Famille': family_name, 
                'Sous famille': sub_family_name,
                'Genre': genus_name, 
                'Espèce': species_name}
    return new_data

In [14]:
data = get_classification('Apis mellifera')
tableau = pd.concat([tableau, pd.DataFrame(data, index=[0])], ignore_index=True)
tableau

,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera


In [15]:
data = get_classification('Bombus')
tableau = pd.concat([tableau, pd.DataFrame(data, index=[0])], ignore_index=True)
tableau

,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,


In [16]:
data = get_classification('Tettigonioidea')
tableau = pd.concat([tableau, pd.DataFrame(data, index=[0])], ignore_index=True)
tableau

,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,
2,Orthoptera,Tettigonioidea,,,,


In [17]:
data = get_classification('Neomyia')
tableau = pd.concat([tableau, pd.DataFrame(data, index=[0])], ignore_index=True)
tableau

,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,
2,Orthoptera,Tettigonioidea,,,,
3,Diptera,,Muscidae,Muscinae,Neomyia,


In [18]:
data = get_classification('Calliphora')
tableau = pd.concat([tableau, pd.DataFrame(data, index=[0])], ignore_index=True)
tableau

,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,Hymenoptera,Apoidea,Apidae,Apinae,Bombus,
2,Orthoptera,Tettigonioidea,,,,
3,Diptera,,Muscidae,Muscinae,Neomyia,
4,Diptera,,Calliphoridae,Calliphorinae,Calliphora,


In [19]:
def get_hierarchie(name):
    """
    """
    # Enregistrement du nom de l'espèce
    nom = Ids(name)
    nom.itis()
    # Extraction de l'id de l'espèce
    for item in nom.ids[nom.name[0]]:
        if item['name'] == nom.name[0]:
            id = item['id']
            break
    # Obtenir la classification de l'espèce
    return hierarchy_full(id, as_dataframe=True)

In [20]:
df_neom = get_hierarchie('Cucujidae')

In [21]:
df_call = get_hierarchie('Cryptophagidae')

In [22]:
intersection_df = df_neom.merge(df_call, how='inner')
intersection_df

,author,parentName,parentTsn,rankName,taxonName,tsn
0,None,,,Kingdom,Animalia,202423
1,None,Animalia,202423,Subkingdom,Bilateria,914154
2,None,Bilateria,914154,Infrakingdom,Protostomia,914155
3,None,Protostomia,914155,Superphylum,Ecdysozoa,914158
4,None,Ecdysozoa,914158,Phylum,Arthropoda,82696
5,None,Arthropoda,82696,Subphylum,Hexapoda,563886
6,None,Hexapoda,563886,Class,Insecta,99208
7,None,Insecta,99208,Subclass,Pterygota,100500
8,None,Pterygota,100500,Infraclass,Neoptera,563890
9,None,Neoptera,563890,Superorder,Holometabola,914213


In [ ]:
last_taxon_name = intersection_df.iloc[-1]['taxonName']
print(last_taxon_name)


### Remplacer :
- Les Mouches difficiles à déterminer par **Diptera**
- Les Coléoptères difficiles à déterminer par **Coleoptera**
- Les Moustiques, Tipules et autres diptères Nématocères par **Nematocera**
- Les Chenilles et fausses-Chenilles par **Lepidoptera**
- Les Syrphes difficiles à déterminer par **Syrphinae**
- Les Terebrants Chalcidiens et autres par **Chalcidoidea**
- Les Epeires et autres Araneidae par **Araneidae**
- Les Punaises difficiles à déterminer par **Heteroptera**
- Les Syrphes aux fémurs enflés par **Syrphinae**
- Les Tachinaires difficiles à déterminer par **Tachinidae**
- Les Punaises prédatrices ternes par **Lygus lineolaris** (Punaise terne)

In [45]:
tableau = pd.DataFrame(columns=['Ordre', 'Super famille', 'Famille', 'Sous famille', 'Genre', 'Espèce'])

def get_id(name):
    """
    Fonction permettant d'obtenir l'id d'une espèce à partir de son nom
    """
    # Enregistrement du nom de l'espèce
    nom = Ids(name)
    nom.itis()
    # Extraction de l'id de l'espèce
    for item in nom.ids[nom.name[0]]:
        if item['name'] == nom.name[0]:
            id = int(item['id'])
            break
    return id

def get_hierarchie(id):
    return hierarchy_full(id, as_dataframe=True)

def get_famille(name, hierarchie):
    """
    """
    # Obtention de l'ordre, de la famille, du genre et de l'espèce
    order_name, super_family_name, family_name, sub_family_name, genus_name, species_name = '', '', '', '', '', ''
    for item in hierarchie:
        if item['parentName'] != name:
            if item['rankName'] == 'Order':
                order_name = item['taxonName']
            elif item['rankName'] == 'Superfamily':
                super_family_name = item['taxonName']
            elif item['rankName'] == 'Family':
                family_name = item['taxonName']
            elif item['rankName'] == 'Subfamily':
                sub_family_name = item['taxonName']
            elif item['rankName'] == 'Genus':
                genus_name = item['taxonName']
            elif item['rankName'] == 'Species':
                species_name = item['taxonName']
        else:
            break
    # Création d'un tableau avec les informations
    new_data = {'Ordre': order_name, 
                'Super famille': super_family_name,
                'Famille': family_name, 
                'Sous famille': sub_family_name,
                'Genre': genus_name, 
                'Espèce': species_name}
    return new_data

def get_classification(name):
    """
    Fonction permettant d'obtenir la classification d'une espèce
    """
    # Obtention de l'id de l'espèce
    id = get_id(name)
    # Obtention de la hiérarchie de l'espèce
    hierarchie = get_hierarchie(id).to_dict('records')
    # Obtention de la famille de l'espèce
    classification = get_famille(name, hierarchie)
    return classification

def get_classification_double(name1, name2):
    """
    Fonction permettant d'obtenir la classification lorsque deux
    espèces sont indiquées
    """
    # Obtention de l'id de l'espèce
    id1 = get_id(name1)
    id2 = get_id(name2)
    # Obtention de la hiérarchie de l'espèce
    hierarchie1 = get_hierarchie(id1)
    hierarchie2 = get_hierarchie(id2)
    hierarchie = hierarchie1.merge(hierarchie2, how='inner')
    last_name = hierarchie.iloc[-1]['taxonName']
    hierarchie = hierarchie.to_dict('records')
    # Obtention de la famille de l'espèce
    classification = get_famille(last_name, hierarchie)
    return classification

In [24]:
name = 'Apis mellifera'
id = get_id(name)
hierarchie = get_hierarchie(id)
hierarchie = hierarchie.to_dict('records')
famille = get_famille(name, hierarchie)
famille

154396

In [42]:
get_classification(name)

{'Ordre': 'Hymenoptera',
 'Super famille': 'Apoidea',
 'Famille': 'Apidae',
 'Sous famille': 'Apinae',
 'Genre': 'Apis',
 'Espèce': 'Apis mellifera'}

In [46]:
get_classification_double('Cucujidae', 'Cryptophagidae')

{'Ordre': 'Coleoptera',
 'Super famille': 'Cucujoidea',
 'Famille': '',
 'Sous famille': '',
 'Genre': '',
 'Espèce': ''}

In [64]:
def unique_df_values(df, column):
    """
    Fonction permettant d'obtenir les valeurs uniques d'un dataframe
    """
    # Obtention des valeurs uniques de la colonne d'intérêt
    unique = df[column].unique()
    unique = list(unique)
    # Séparation au niveau des virgules
    for i in range(len(unique)):
        unique[i] = unique[i].replace(' et autres', '')
        unique[i] = unique[i].split(', ')
    return unique

def get_classification_df(df, column):
    """
    Fonction permettant d'obtenir la classification d'un dataframe
    """
    # Obtention des valeurs uniques de la colonne d'intérêt
    unique = unique_df_values(df, column)
    # Obtention de la classification pour chaque espèce
    size = len(unique)
    dictionary = {}
    for i in range(size):
        length = len(unique[i])
        if length == 1:
            print(unique[i][0])
            classification = get_classification(unique[i][0])
            dictionary[unique[i][0]] = classification
        elif length == 2:
            print(unique[i][0], unique[i][1])
            classification = get_classification_double(unique[i][0], unique[i][1])
            dictionary[unique[i][0]] = classification
        else:
            print('Erreur')
    return dictionary

In [70]:
df = pd.read_csv('insect_sc.csv')
unique = unique_df_values(df, 'x')
print(len(unique))

2183


In [86]:
unique[:20]

[['Apis mellifera'],
 ['Bombus'],
 ['Sarcophaga'],
 ['Tettigonoidea'],
 ['Neomyia', 'Calliphora'],
 ['Diptera'],
 ['Tephritidae'],
 ['Eupeodes', 'Scaeva'],
 ['Episyrphus balteatus'],
 ['Meliscaeva'],
 ['Eristalis'],
 ['Coleoptera'],
 ['Geomyza tripunctata'],
 ['Nematocera'],
 ['Chironomidae'],
 ['Trichoceridae'],
 ['Pyrrhocoris apterus'],
 ['Thysanoptera'],
 ['Piezodorus lituratus'],
 ['Formicinae']]

In [105]:
df = df[:10]

In [87]:
df.head(20)

,x
0,Apis mellifera
1,Bombus
2,Bombus
3,Sarcophaga
4,Apis mellifera
5,Tettigonioidea
6,"Neomyia, Calliphora et autres"
7,Diptera
8,Apis mellifera
9,Diptera


In [94]:
df = df.replace('Tettigonoidea', 'Tettigonioidea')
df = df.replace('Episyrphus balteatus', 'Syrphus')
df.head(20)

,x
0,Apis mellifera
1,Bombus
2,Bombus
3,Sarcophaga
4,Apis mellifera
5,Tettigonioidea
6,"Neomyia, Calliphora et autres"
7,Diptera
8,Apis mellifera
9,Diptera


In [108]:
classif = get_classification_df(df, 'x')

Apis mellifera
Bombus
Sarcophaga
Tettigonioidea
Neomyia Calliphora
Diptera


In [109]:
df['classification'] = df['x'].apply(lambda x: classif[x] if x in classif else None)
df

,x,classification
0,Apis mellifera,"{'Ordre': 'Hymenoptera', 'Super famille': 'Apo..."
1,Bombus,"{'Ordre': 'Hymenoptera', 'Super famille': 'Apo..."
2,Bombus,"{'Ordre': 'Hymenoptera', 'Super famille': 'Apo..."
3,Sarcophaga,"{'Ordre': 'Diptera', 'Super famille': '', 'Fam..."
4,Apis mellifera,"{'Ordre': 'Hymenoptera', 'Super famille': 'Apo..."
5,Tettigonioidea,"{'Ordre': 'Orthoptera', 'Super famille': 'Tett..."
6,"Neomyia, Calliphora et autres",None
7,Diptera,"{'Ordre': 'Diptera', 'Super famille': '', 'Fam..."
8,Apis mellifera,"{'Ordre': 'Hymenoptera', 'Super famille': 'Apo..."
9,Diptera,"{'Ordre': 'Diptera', 'Super famille': '', 'Fam..."


In [110]:
df['Ordre'] = df['classification'].apply(lambda x: x['Ordre'] if isinstance(x, dict) and 'Ordre' in x else None)
df['Super famille'] = df['classification'].apply(lambda x: x['Super famille'] if isinstance(x, dict) and 'Super famille' in x else None)
df['Famille'] = df['classification'].apply(lambda x: x['Famille'] if isinstance(x, dict) and 'Famille' in x else None)
df['Sous famille'] = df['classification'].apply(lambda x: x['Sous famille'] if isinstance(x, dict) and 'Sous famille' in x else None)
df['Genre'] = df['classification'].apply(lambda x: x['Genre'] if isinstance(x, dict) and 'Genre' in x else None)
df['Espèce'] = df['classification'].apply(lambda x: x['Espèce'] if isinstance(x, dict) and 'Espèce' in x else None)
df

,x,classification,Ordre,Super famille,Famille,Sous famille,Genre,Espèce
0,Apis mellifera,"{'Ordre': 'Hymenoptera', 'Super famille': 'Apo...",Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
1,Bombus,"{'Ordre': 'Hymenoptera', 'Super famille': 'Apo...",Hymenoptera,Apoidea,Apidae,Apinae,Bombus,
2,Bombus,"{'Ordre': 'Hymenoptera', 'Super famille': 'Apo...",Hymenoptera,Apoidea,Apidae,Apinae,Bombus,
3,Sarcophaga,"{'Ordre': 'Diptera', 'Super famille': '', 'Fam...",Diptera,,Sarcophagidae,Sarcophaginae,Sarcophaga,
4,Apis mellifera,"{'Ordre': 'Hymenoptera', 'Super famille': 'Apo...",Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
5,Tettigonioidea,"{'Ordre': 'Orthoptera', 'Super famille': 'Tett...",Orthoptera,Tettigonioidea,,,,
6,"Neomyia, Calliphora et autres",None,None,None,None,None,None,None
7,Diptera,"{'Ordre': 'Diptera', 'Super famille': '', 'Fam...",Diptera,,,,,
8,Apis mellifera,"{'Ordre': 'Hymenoptera', 'Super famille': 'Apo...",Hymenoptera,Apoidea,Apidae,Apinae,Apis,Apis mellifera
9,Diptera,"{'Ordre': 'Diptera', 'Super famille': '', 'Fam...",Diptera,,,,,
